In [1]:
import pandas as pd
import numpy as np
from math import pi
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from tqdm import tqdm_notebook as tqdm

from numpy import linalg as LA

init_notebook_mode(connected=True)

In [2]:
def generate_trajectory(X0, V0, T = 2, N = 500, bias = 0, sigma_acc = 0.3):

    V = np.zeros((N, 2))
    X = np.zeros((N, 2))

    X[0] = X0
    V[0] = V0
    
    a = np.random.normal(loc=bias, scale=sigma_acc, size=(N,2))

    V[1:] = a[:-1]*T
    V = np.cumsum(V, axis=0)
    
    X[1:] = V[:-1]*T + a[:-1]*(T**2)/2
    X = np.cumsum(X, axis=0)
    
    S = np.zeros((N, 4))
    S[:,0] = X[:,0]
    S[:,1] = V[:,0]
    S[:,2] = X[:,1]
    S[:,3] = V[:,1]
    
    return X, V, S

In [43]:
X, V, S = generate_trajectory(X0=[1000, 1000], V0=[100, 100], T = 2, N = 500, sigma_acc=0.3)

In [44]:
data = [
    go.Scatter(
        y=X[:,1],
        x=X[:,0],
        name='true x'
    ),
]

layout= go.Layout(
    title= 'Kalman filtration of gapped data',
    xaxis= dict(
        title= 'Step',
    ),
    yaxis=dict(
        title= 'X',
    ),
    showlegend= True
)
fig= go.Figure(data=data, layout=layout)
iplot(fig)

In [45]:
def generate_polar(X, Y, N):
        
    D = np.zeros((N))
    beta = np.zeros((N))
    
    D[:] = np.sqrt(X[:]**2 + Y[:]**2)
    beta[:] = np.arctan2(X[:], Y[:])
    
    return D, beta

In [46]:
trD, trbeta = generate_polar(X[:,0], X[:,1], N = 500)

In [47]:
def measure(D, beta, N, sigma_D = 50, sigma_b = 0.004, sigma_b_add = 0.001, mean = 0):
    
    msr_D = D + np.random.normal(loc = mean, scale = sigma_D, size = N)
    msr_b = beta + np.random.normal(loc = mean, scale = sigma_b, size = N)

    msr_D[1::2] = None
    msr_b[1::2] = beta[1::2] + np.random.normal(loc = mean, scale = sigma_b_add, size = len(msr_b[1::2]))
    
    return msr_D, msr_b

In [48]:
msr_D, msr_b = measure(trD, trbeta, N = 500)

In [49]:
msr_X = msr_D*np.sin(msr_b)
msr_Y = msr_D*np.cos(msr_b)

In [50]:
T = 1
F = np.array([[1, T, 0, 0], [0, 1, 0, 0], [0, 0, 1, T], [0, 0, 0, 1]])
G = np.array([[(T**2)/2, 0], [T, 0], [0, (T**2)/2], [0, T]])

In [51]:
S0 = np.array((msr_D[2]*np.sin(msr_b[2]), (msr_D[2]*np.sin(msr_b[2]) - msr_D[0]*np.sin(msr_b[0]))/(2*T), msr_D[2]*np.cos(msr_b[2]), (msr_D[2]*np.cos(msr_b[2]) - msr_D[0]*np.cos(msr_b[0]))/(2*T)))
P0 = np.eye(4)*(10**10)

In [190]:
def H1(X):
    return np.array([np.sqrt(X[0]**2 + X[2]**2), np.math.atan(X[0]/X[2])])

def H2(X):
    return np.math.atan(X[0]/X[2])

def dH1(X):
    return np.array([[X[0]/np.sqrt(X[0]**2 + X[2]**2), 0, X[2]/np.sqrt(X[0]**2 + X[2]**2), 0],
                                 [X[2]/(X[0]**2 + X[2]**2), 0, -X[0]/(X[0]**2 + X[2]**2), 0]])

def dH2(X):
    return np.array([X[2]/(X[0]**2 + X[2]**2), 0, -X[0]/(X[0]**2 + X[2]**2), 0])


def kalman(msr, S0, P0, T=2, sigma_D = 50, sigma_b = 0.004, sigma_b_add = 0.001, K=None, q=0):
    
    size = msr.shape
    
    F = np.array([[1, T, 0, 0], [0, 1, 0, 0], [0, 0, 1, T], [0, 0, 0, 1]])
    G = np.array([[(T**2)/2, 0], [T, 0], [0, (T**2)/2], [0, T]])
    
    Q = G.dot(G.T)*0.3**2
    
    P = np.zeros((size[0], 4, 4))
    P_ = np.zeros((size[0], 4, 4))
    
    R1 = np.array([[sigma_D**2, 0], [0, sigma_b**2]])
    R2 = sigma_b_add**2
    
    K = np.zeros((size[0], 4, 2))
    
    D = np.zeros(len(msr)) # predicted 
    D_ = np.zeros(len(msr)) # filtered
    beta = np.zeros(len(msr)) # predicted 
    beta_ = np.zeros(len(msr)) # filtered
    
    # allocate space for arrays
    X=np.zeros((size[0], 4))
    X_=np.zeros((size[0], 4))
    
    # intial guesses
    X_[0] = X_[1] = X_[2] = X_[3] = S0
    P_[0] = P_[1] = P_[2] = P_[3] = P0
    
    for k in range(4, size[0]):
        # prediction
        X[k] = F.dot(X_[k-1])
        P[k] = F.dot(P_[k-1].dot(F.T)) + Q
        
        # polar predicted
        D[k] = np.sqrt(X[k][0]**2 + X[k][2]**2) # X and Y from state S0
        beta[k] = np.arctan2(X[k][0], X[k][2])
        
#         if not np.isnan(msr[k,0]):
#             dH_first = np.array([[X[k,0]/np.sqrt(X[k,0]**2 + X[k,2]**2), 0, X[k,2]/np.sqrt(X[k,0]**2 + X[k,2]**2), 0],
#                                  [X[k,2]/(X[k,0]**2 + X[k,2]**2), 0, -X[k,0]/(X[k,0]**2 + X[k,2]**2), 0]])

#             K[k] = P[k].dot(dH_first.T.dot(np.linalg.inv(dH_first.dot(P[k].dot(dH_first.T)) + R_first)))
#             X_[k] = X[k] + K[k].dot(msr[k] - H_first(X[k]))
#             P_[k] = (np.eye(4) - np.dot(K[k], dH_first)).dot(P[k])
            
        
#         else:
#             dH_second = np.array([X[k,2]/(X[k,0]**2 + X[k,2]**2), 0, -X[k,0]/(X[k,0]**2 + X[k,2]**2), 0])

#             #K[k] = P[k].dot(dH_second.T.dot(np.linalg.inv(dH_second.dot(P[k].dot(dH_second.T)) + R_second)))
#             K[k,:,1] = (P[k].dot(dH_second.T))/(dH_second.dot(P[k].dot(dH_second.T)) + R_second)
#             K[k,:,0] = K[k-1,:,0]
            
        
#             X_[k] = X[k] + K[k]*(msr[k][1] - H_second(X[k]))
#             P_[k] = (np.eye(4) - np.dot(K[k],dH_second)).dot(P[k])  
            
        if not np.isnan(msr[k,0]):    
            dH = dH1(X[k])
            R = R1
            m = msr[k]
            H = H1(X[k])
        else:
            dH = dH2(X[k])
            R = R2
            m = msr[k][1]
            H = H2(X[k])
            
        K[k] = P[k].dot(dH.T.dot(np.linalg.inv(dH.dot(P[k].dot(dH.T)) + R)))   
        X_[k] = X[k] + K[k]*(m - H)
        P_[k] = (np.eye(4) - np.dot(K[k], dH)).dot(P[k])  
            
        # polar filtered
        D_[k] = np.sqrt(X_[k][0]**2 + X_[k][2]**2) # X and Y from state S0
        beta_[k] = np.arctan2(X_[k][0], X_[k][2])    
            
#     return X, X_, np.sqrt(P[:,0,0]), np.sqrt(P_[:,0,0]), np.linalg.matrix_power(F,7).dot(X_.T)[0]
    return D, beta, D_, beta_, X_, K



    N = len(Zm)
    X = []
    Xp.append(X0)
    Xp.append(X0)
    Xp.append(X0)
    X_ = []
    X_.append(X0)
    X_.append(X0)
    X_.append(X0)
    I = np.eye(4)
    K = []
    Pp = []
    Pp.append(P00)
    Pp.append(P00)
    Pp.append(P00)
    Pf = []
    Pf.append(P00)
    Pf.append(P00)
    Pf.append(P00)
    for i in range(2,N-1):
        # prediction
        Xp.append(np.matmul(F, Xf[i]))
        Pp.append(np.matmul(np.matmul(F, Pf[i]), Ft) + Q)
        # filtration
        if is_odd(i):    
            dh_dX = dh_d1(Xp[i+1])
            R = R1
            z = Zm[i+1]
            h = h1(Xp[i+1])
        else:
            dh_dX = dh_d2(Xp[i+1])
            R = R2
            z = Zm[i+1][1]
            h = h2(Xp[i+1])
        dh_dX_T = np.transpose(dh_dX)
        K.append(np.matmul(Pp[i+1], np.matmul(dh_dX_T, np.linalg.inv(np.matmul(np.matmul(dh_dX, Pp[i+1]), dh_dX_T) + R))))
        Xf.append(Xp[i+1] + (np.matmul(K[-1], z - h)).reshape((4,1)))
        Pf.append(np.matmul(I - np.matmul(K[-1], dh_dX), Pp[i+1]))
    return [Xf, Xp, Pf, Pp];

In [191]:
msr = np.zeros((len(msr_b), 2))
msr[:,0] = msr_D
msr[:,1] = msr_b
filtered = kalman(msr, S0=S0, P0=P0)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [102]:
data = [
    go.Scatter(
        x=filtered[2],
        y=filtered[3],
        name='filtered trajectory'
    ),
    go.Scatter(
        x=trD,
        y=trbeta,
        name='true trajectory'
    ),
    go.Scatter(
        x=msr_D,
        y=msr_b,
        name='measured trajectory',
        mode='markers'
    ),
#     go.Scatter(
#         x=filtered[4][:,0],
#         y=filtered[4][:,2],
#         name='filtered trajectory'
#     ),
]

layout= go.Layout(
    title= 'C',
    xaxis= dict(
        title= 'Step',
    ),
    yaxis=dict(
        title= 'X',
    ),
    showlegend= True
)
fig= go.Figure(data=data, layout=layout)
iplot(fig)

In [156]:
def experiment(M = 20, N = 500):
    
    # allocate space for arrays
    eD = np.zeros(N)
    eD_ = np.zeros(N)
    ebeta = np.zeros(N)
    ebeta_ = np.zeros(N)
    
    for i in tqdm(range(M)):

        X, V, S = generate_trajectory(X0=[1000, 2000], V0=[10, 10])
        trD, trbeta = generate_polar(X[:,0], X[:,1], N = 500)
        msr_D, msr_b = measure(trD, trbeta, N = 500)
        msr_X = msr_D*np.sin(msr_b)
        msr_Y = msr_D*np.cos(msr_b)
        
        #z = np.column_stack((msr_X,msr_Y))
        z = np.column_stack((msr_D, msr_b))
        D, beta, D_, beta_, X_, K = kalman(z, S0=S0, P0=P0)
        
        eD_ += (D_ - trD)**2
        ebeta_ += (beta_ - trbeta)**2
        
        eD += (D - trD)**2
        ebeta += (beta - trbeta)**2
        
    eD_ = np.sqrt(eD_/M)
    ebeta_ = np.sqrt(ebeta_/M)
    
    eD = np.sqrt(eD/(M-1))
    ebeta = np.sqrt(ebeta/M)
    
    return eD, eD_, ebeta, ebeta_

In [160]:
eD, eD_, ebeta, ebeta_ = experiment()

In [161]:
N = 500
sigma_b = np.full((N), 0.004)
sigma_D = np.full((N), 50)

In [162]:
data = [
    go.Scatter(
        y = eD[2:400],
        name='std of prediction error D'
    ),
    go.Scatter(
        y = eD_[2:400],
        name='std of filtration error'
    ),
    go.Scatter(
        y = sigma_D,
        name='sigma_D',
    ),
]

layout= go.Layout(
    title= '(a) Errors of extrapolation and filtration estimates of range 𝐷 relative to 𝜎𝐷',
    xaxis= dict(
        title= 'Step',
    ),
    yaxis=dict(
        title= 'error',
    ),
    showlegend= True
)
fig= go.Figure(data=data, layout=layout)
iplot(fig)